In [1]:
BRANCH='main'
import os
import wget

In [ ]:
"""
You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

Instructions for setting up Colab are as follows:
1. Open a new Python 3 notebook.
2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
4. Run this cell to set up dependencies.
"""
# If you're using Google Colab and not running locally, run this cell

# install NeMo
!python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[nlp]

### Introduction

In this notebook we demonstrate how to use NeMo's implementation of LoRA (Low Rank Adaptation) for fine-tuning large language models. Our implementation is based on the [paper](https://openreview.net/pdf?id=nZeVKeeFYf9) by Hu et al.

We are going to show you how to:
    
    1. Train a LoRA model on a simple Extractive QA task.
    2. Inspect the trained LoRA model showing the parameters it contains.
    3. Run inference with the based model with the LoRA parameters.
    4. Merge the LoRA parameters into the base model and run inference again on the merged model.

In this tutorial we will be focusing on LoRA, but the training and evaluation methods described here will be applicable for other Parameter-efficient Fine tuning (PEFT) methods in NeMo.

### Tasks and Datasets
We will be using LoRA to teach our GPT model to do Extractive Question Answering.

We will be using the [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) reading comprehension dataset, consisting of questions posed by crowd workers on a set of Wikipedia articles, where the answer to every question is a segment of text. More information on [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/) can be found on their website or in their paper by Rajpurkar et. al "[Know What You Don’t Know: Unanswerable Questions for SQuAD](https://arxiv.org/pdf/1806.03822.pdf)".

LoRA (and all PEFT tuning) models expect at least two fields in the jsonl files. The `input` field should contain all the tokens necessary for the model to generate the `output`. For example for extractive QA, the `input` should contain the context text as well as the question.

```
[
    {"input": "User: Context: [CONTEXT_1] Question: [QUESTION_1]\n\nAssistant:", "output": [ANSWER_1]},
    {"input": "User: Context: [CONTEXT_2] Question: [QUESTION_2]\n\nAssistant:", "output": [ANSWER_2]},
    {"input": "User: Context: [CONTEXT_3] Question: [QUESTION_3]\n\nAssistant:", "output": [ANSWER_3]},
]
```
Note that we use keywords in the input like `Context:`, `Question:` to separate the text representing the context and question. We also use the keyword `User:` and end each of the input with `\n\nAssistant:` tokens. These are recommended because NeMo's instruction-tuned models are trained with a prefix of `User:` and suffix `\n\nAssistant:`.

In [2]:
# You can replace DATA_DIR and NEMO_DIR with your own locations
DATA_DIR = "data"
NEMO_DIR = "."
os.makedirs(DATA_DIR, exist_ok=True)


For each dataset we have preprocessing scripts pre-written in NeMo's example directory located in `examples/nlp`. Let's download those now. 

In [3]:
# download the preprocessing scripts from github for the purpose of this tutorial
! wget -nc https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/scripts/dataset_processing/nlp/squad/prompt_learning_squad_preprocessing.py

File ‘prompt_learning_squad_preprocessing.py’ already there; not retrieving.



Now let's down load and process the dataset.

In [4]:
SQUAD_DIR = os.path.join(DATA_DIR, "SQuAD")
os.makedirs(SQUAD_DIR, exist_ok=True)

# Download the SQuAD dataset
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
!wget -nc https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
!mv train-v1.1.json {SQUAD_DIR}
!mv dev-v1.1.json {SQUAD_DIR}

--2023-05-27 09:45:11--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 30288272 (29M) [application/json]
Saving to: ‘train-v1.1.json’

train-v1.1.json     100%[===================>]  28.88M  80.2MB/s    in 0.4s    

2023-05-27 09:45:13 (80.2 MB/s) - ‘train-v1.1.json’ saved [30288272/30288272]

--2023-05-27 09:45:13--  https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v1.1.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4854279 (4.6M) [application/json]
Saving to: ‘dev-v1.1.json’

dev-v1.1.json       

In [ ]:
# Preprocess squad data
!python prompt_learning_squad_preprocessing.py --sft-format --data-dir {SQUAD_DIR}

In [6]:
# What the squad dataset looks like after processing
! head -200 $SQUAD_DIR/squad_train.jsonl > $SQUAD_DIR/squad_short_train.jsonl
! head -20 $SQUAD_DIR/squad_val.jsonl > $SQUAD_DIR/squad_short_val.jsonl
! head -4 $SQUAD_DIR/squad_short_val.jsonl
! head -4 $SQUAD_DIR/squad_short_train.jsonl


{"input": "User: Context:Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24\u201310 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the \"golden anniversary\" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as \"Super Bowl L\"), so that the logo could prominently feature the Arabic numerals 50. Question:Which NFL team represented the AFC at Super Bowl 50?\n\nAssistant:", "output": "Denver Broncos"}
{"input": "User: Context:Super Bowl 50 was an American football game to determine th

### Model Config Setup
Now we will begin setting up the config file needed for PEFT tuning. We use a single config for all supported PEFT methods (LoRA, Adapter and P-Tuning).  All PEFT methods use classes defined in  [megatron_gpt_peft_models.py](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/nlp/models/language_modeling/megatron_gpt_peft_models.py). All PEFT Classes inherit from `MegatronGPTSFTModel` which is the class that governs instruction tuning.

In [5]:
from omegaconf import OmegaConf

CONFIG_DIR = os.path.join(NEMO_DIR, "conf")
os.makedirs(CONFIG_DIR, exist_ok=True)

# Download the example config file
wget.download(f'https://raw.githubusercontent.com/NVIDIA/NeMo/{BRANCH}/examples/nlp/language_modeling/tuning/conf/megatron_gpt_peft_tuning_config.yaml', CONFIG_DIR)

# Load the example config file so we can start editing it
CONFIG_PATH = os.path.join(CONFIG_DIR, "megatron_gpt_peft_tuning_config.yaml")
config = OmegaConf.load(CONFIG_PATH)

The `config` contains several attributes required by the `MegatronGPTPEFTModel`. First we will set the training data path and the validation data path in the config.
The `config` allows us to set a list of `jsonl` files as training files and sample examples from each file with different probabilities. For simplicity we are going to use just one training file and thus the sampling probability is set to `1.0`

We can also monitor validation loss from multiple validation files during training. Again for simplicity we will use just one validation file.

In [7]:
config.model.data.train_ds.file_names = [f"{SQUAD_DIR}/squad_short_train.jsonl"]
config.model.data.train_ds.concat_sampling_probabilities=[1.0]
config.model.data.validation_ds.file_names = [f"{SQUAD_DIR}/squad_short_val.jsonl"]
config.model.data.validation_ds.names=["squad_val"]

### PEFT Config
The attribute [config.model.peft](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/language_modeling/tuning/conf/megatron_gpt_peft_tuning_config.yaml#L78) contains settings that control the PEFT training method and its related hyperpameters. We currently support `lora`, `adapters`, `ptuning` and `ia3`. We can instruct the training script to use one of these methods by setting the config.model.peft.peft_scheme attribute.

The other hyperparams associated with lora tuning are present in the [config.model.peft.lora_tuning](https://github.com/NVIDIA/NeMo/blob/main/examples/nlp/language_modeling/tuning/conf/megatron_gpt_peft_tuning_config.yaml#L92) attribute.

In [8]:
config.model.peft.peft_scheme="lora"  # we can also set this to adapter or ptuning or ia3
print(OmegaConf.to_yaml(config.model.peft.lora_tuning))

adapter_dim: 32
adapter_dropout: 0.0
column_init_method: xavier
row_init_method: zero



**Note:** In the original LoRA paper each attention projection (`K`, `Q`, `V` and `O`) can have their own Low-Rank projections. However, NeMo's attention implementation fuses `KQV` into a single projection and thus our LoRA implementation learns a single Low-Rank projection for `KQV` in a combined fashion. We do not support LoRA for the `O` matrix at this point.

### Prompt Formatting
The `config.model.data.train_ds.prompt_template` attribute allows us to further tweak the format of the input and output if needed. In this example, we have "encoding" our format inside the `jsonl` file directly. So we can keep the `prompt_template` in the config simple.(See previous section on Data Preparation). 

In [9]:
config.model.data.train_ds.prompt_template ="{input} {output}"

### Setting the Pretrained GPT Model
Next we will set the "base language model" upon which we will perform LoRA tuning. Obviously, larger base models will have better performance on downstream tasks but for the purposes of this tutorial we will use a small 345M parameter GPT model.

In [10]:
# Check what GPT .nemo models we have available on NGC
from nemo.collections.nlp.models.language_modeling.megatron_gpt_model import MegatronGPTModel
megatron_gpt_345m_nemo_url = MegatronGPTModel.list_available_models()[0].location
megatron_gpt_345m_nemo_url # should point to the 345m megatron gpt model '.nemo' file

[NeMo W 2023-05-27 09:45:41 experimental:27] Module <class 'nemo.collections.nlp.models.text_normalization_as_tagging.thutmose_tagger.ThutmoseTaggerModel'> is experimental, not ready for production and is not fully supported. Use at your own risk.
[NeMo W 2023-05-27 09:45:41 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.


'https://api.ngc.nvidia.com/v2/models/nvidia/nemo/megatron_gpt_345m/versions/1/files/megatron_gpt_345m.nemo'

If we wanted to use the GPT model class directly, we could instantiate a trainer then download the model by calling running 
`gpt_model = MegatronGPTModel.from_pretrained(model_name="megatron_gpt_345m", trainer=trainer).cuda()`. But we just need the `.nemo` file in our working NeMo directory in this tutorial, so we will download it using `wget`. 

In [11]:
# Download the model from NGC
gpt_file_name = "megatron_gpt_345m.nemo"
!wget  -nc --content-disposition {megatron_gpt_345m_nemo_url} -O {NEMO_DIR}/{gpt_file_name}

File ‘./megatron_gpt_345m.nemo’ already there; not retrieving.


Now that we have a `.nemo` GPT file to work with. We need to add its path in our prompt learning config. 

In [12]:
# Set GPT model path on prompt learning config
config.model.restore_from_path = gpt_file_name

Next, we will set where we want to save all the intermediate training logs and checkpoints. As well as other training settings such as: number of training steps, batch size and validation check interval, and num_workers for data processing.

In [13]:
config.exp_manager.exp_dir=f"{NEMO_DIR}/peft_lora"
config.exp_manager.explicit_log_dir="training_info"
config.trainer.max_steps=100
config.model.micro_batch_size=1
config.model.global_batch_size=4
config.trainer.val_check_interval=50
config.model.data.train_ds.num_workers=0  # 0 is recommended which just uses the main thread to process training examples
config.model.data.validation_ds.num_workers=0 # 0 is recommended which just uses the main thread to process the validation examples

Let's have a look at all the values we've set in the model config. You can change any of these values in the same manner we've been using above. 

In [14]:
# Final model config
print(OmegaConf.to_yaml(config.model))

seed: 1234
tensor_model_parallel_size: 1
pipeline_model_parallel_size: 1
global_batch_size: 4
micro_batch_size: 1
restore_from_path: megatron_gpt_345m.nemo
resume_from_checkpoint: null
save_nemo_on_validation_end: false
sync_batch_comm: false
megatron_amp_O2: false
sequence_parallel: false
activations_checkpoint_granularity: null
activations_checkpoint_method: null
activations_checkpoint_num_layers: null
answer_only_loss: true
gradient_as_bucket_view: false
hidden_dropout: 0.0
attention_dropout: 0.0
ffn_dropout: 0.0
peft:
  peft_scheme: lora
  restore_from_path: null
  adapter_tuning:
    type: parallel_adapter
    adapter_dim: 32
    adapter_dropout: 0.0
    norm_position: pre
    column_init_method: xavier
    row_init_method: zero
    norm_type: mixedfusedlayernorm
  lora_tuning:
    adapter_dim: 32
    adapter_dropout: 0.0
    column_init_method: xavier
    row_init_method: zero
  p_tuning:
    virtual_tokens: 10
    bottleneck_dim: 1024
    embedding_dim: 1024
    init_std: 0.023


# Building the PyTorch Lightning Trainer
NeMo models are primarily PyTorch Lightning modules - and therefore are entirely compatible with the PyTorch Lightning ecosystem.

Let's first instantiate a Trainer object

In [15]:
import torch
import pytorch_lightning as pl
from nemo.collections.nlp.parts.nlp_overrides import NLPDDPStrategy
from pytorch_lightning.plugins.environments import TorchElasticEnvironment

# let's modify some trainer configs
# check if we have GPU available and uses it
accelerator = 'gpu' if torch.cuda.is_available() else 'cpu'
config.trainer.accelerator = accelerator
config.trainer.devices = 1
config.trainer.max_epochs = 4
config.trainer.val_check_interval = 1.0

# for PyTorch Native AMP set precision=16
config.trainer.precision = 16 if torch.cuda.is_available() else 32

# setup cluster environment parameters"
# use torch elastic cluster environment so `create_process_externally` is True
# the launcher is set to None. It will not try to spawn new processes.
# It won't create the misconfiguration error because of the `interactive session`
os.environ["LOCAL_RANK"] = '0'
os.environ["RANK"] = '0'
os.environ["WORLD_SIZE"] = '1'

strategy = NLPDDPStrategy(find_unused_parameters=False, no_ddp_communication_hook=True)
plugins = [TorchElasticEnvironment()]
trainer = pl.Trainer(plugins= plugins, strategy=strategy, **config.trainer)

print("Trainer config - \n")
print(OmegaConf.to_yaml(config.trainer))

Using 16bit None Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
`Trainer(val_check_interval=1.0)` was configured so validation will run at the end of the training epoch..


Trainer config - 

devices: 1
accelerator: gpu
num_nodes: 1
precision: 16
logger: false
enable_checkpointing: false
replace_sampler_ddp: false
max_epochs: 4
max_steps: 100
log_every_n_steps: 10
val_check_interval: 1.0
gradient_clip_val: 1.0



In [16]:
print(OmegaConf.to_yaml(config.exp_manager))

explicit_log_dir: training_info
exp_dir: ./peft_lora
name: ${name}
create_wandb_logger: false
wandb_logger_kwargs:
  project: null
  name: null
resume_if_exists: true
resume_ignore_no_checkpoint: true
create_checkpoint_callback: true
checkpoint_callback_params:
  monitor: validation_${model.data.validation_ds.metric.name}
  save_top_k: 1
  mode: min
  save_nemo_on_train_end: true
  filename: ${name}--{${exp_manager.checkpoint_callback_params.monitor}:.3f}-{step}-{consumed_samples}
  model_parallel_size: ${model.tensor_model_parallel_size}
  always_save_nemo: false
  save_best_model: true
create_early_stopping_callback: true
early_stopping_callback_params:
  monitor: val_loss
  mode: min
  min_delta: 0.001
  patience: 10
  verbose: true
  strict: false



### Setting up a NeMo Experiment

NeMo has an experiment manager that handles logging and checkpointing for us, so let's use it:

In [17]:
from nemo.utils.exp_manager import exp_manager

# Set name of the experiment 
config.name = 'lora_example_tuning'
config.exp_manager.resume_if_exists = False

# Init the experiment manager and view the exp_dir
exp_dir = exp_manager(trainer, config.get("exp_manager", None))
exp_dir = str(exp_dir)
print(exp_dir)

[NeMo E 2023-05-27 09:46:08 exp_manager:646] exp_manager received explicit_log_dir: training_info and at least one of exp_dir: ./peft_lora, or version: None. Please note that exp_dir, name, and version will be ignored.


[NeMo I 2023-05-27 09:46:08 exp_manager:374] Experiments will be logged at training_info
[NeMo I 2023-05-27 09:46:08 exp_manager:797] TensorboardLogger has been set up


[NeMo W 2023-05-27 09:46:08 exp_manager:893] The checkpoint callback was told to monitor a validation value and trainer's max_steps was set to 100. Please ensure that max_steps will run for at least 1 epochs to ensure that checkpointing will not error out.


training_info


### LoRA Training
We now set up the process for training a LoRA model. We first require a config that contains details about the base language model upon which we will train our LoRA model. So we first extract the `base_model_cfg`

In [18]:
from nemo.collections.nlp.models.language_modeling.megatron_gpt_sft_model import MegatronGPTModel
from nemo.collections.nlp.parts.nlp_overrides import NLPSaveRestoreConnector, PEFTSaveRestoreConnector
base_model_save_restore_connector = NLPSaveRestoreConnector()
base_model_cfg = MegatronGPTModel.restore_from(
    restore_path=config.model.restore_from_path,
    trainer=trainer,
    return_config=True,
    save_restore_connector=base_model_save_restore_connector,
    )

[NeMo W 2023-05-27 09:46:30 experimental:27] Module <class 'nemo.collections.nlp.data.language_modeling.megatron.megatron_batch_samplers.MegatronPretrainingRandomBatchSampler'> is experimental, not ready for production and is not fully supported. Use at your own risk.


Next, we update the `base_model_cfg` with any new settings we employ in our current (LoRA) `config`.

In [19]:
from omegaconf.omegaconf import open_dict
from nemo.collections.nlp.models.language_modeling.megatron_gpt_peft_models import MegatronGPTLoRAModel
OmegaConf.set_struct(base_model_cfg, True)
OmegaConf.resolve(config)
with open_dict(base_model_cfg):
    base_model_cfg.megatron_amp_O2 = config.model.get('megatron_amp_O2', False)
    base_model_cfg.micro_batch_size = config.model.data.train_ds.micro_batch_size
    base_model_cfg.global_batch_size = config.model.data.train_ds.global_batch_size
    base_model_cfg.sequence_parallel = config.model.get("sequence_parallel", False)
    base_model_cfg.data = config.model.data
    base_model_cfg.optim = config.model.optim
    base_model_cfg.precision = config.trainer.precision
    base_model_cfg.answer_only_loss = config.model.answer_only_loss
    base_model_cfg.restore_from_path = config.model.restore_from_path
    base_model_cfg.resume_from_checkpoint = config.model.resume_from_checkpoint
    base_model_cfg.save_nemo_on_validation_end = config.model.save_nemo_on_validation_end
    base_model_cfg.peft = config.model.peft
    base_model_cfg.target = f"{MegatronGPTLoRAModel.__module__}.{MegatronGPTLoRAModel.__name__}"

Next, we instantiate the LoRA model class

In [20]:
from nemo.collections.nlp.parts.nlp_overrides import PEFTSaveRestoreConnector
peft_save_restore_connector = PEFTSaveRestoreConnector(
    peft_model_nemo_path=None, peft_model_ckpt_path=None
    )
model = MegatronGPTLoRAModel.restore_from(
    restore_path=config.model.restore_from_path,
    trainer=trainer,
    override_config_path=base_model_cfg,
    save_restore_connector=peft_save_restore_connector,
)

[NeMo I 2023-05-27 09:46:39 megatron_init:232] Rank 0 has data parallel group: [0]
[NeMo I 2023-05-27 09:46:39 megatron_init:235] All data parallel group ranks: [[0]]
[NeMo I 2023-05-27 09:46:39 megatron_init:236] Ranks 0 has data parallel rank: 0
[NeMo I 2023-05-27 09:46:39 megatron_init:244] Rank 0 has model parallel group: [0]
[NeMo I 2023-05-27 09:46:39 megatron_init:245] All model parallel group ranks: [[0]]
[NeMo I 2023-05-27 09:46:39 megatron_init:255] Rank 0 has tensor model parallel group: [0]
[NeMo I 2023-05-27 09:46:39 megatron_init:259] All tensor model parallel group ranks: [[0]]
[NeMo I 2023-05-27 09:46:39 megatron_init:260] Rank 0 has tensor model parallel rank: 0
[NeMo I 2023-05-27 09:46:39 megatron_init:274] Rank 0 has pipeline model parallel group: [0]
[NeMo I 2023-05-27 09:46:39 megatron_init:286] Rank 0 has embedding group: [0]
[NeMo I 2023-05-27 09:46:39 megatron_init:292] All pipeline model parallel group ranks: [[0]]
[NeMo I 2023-05-27 09:46:39 megatron_init:293]

[NeMo W 2023-05-27 09:46:39 modelPT:244] You tried to register an artifact under config key=tokenizer.vocab_file but an artifact for it has already been registered.


[NeMo I 2023-05-27 09:46:39 tokenizer_utils:204] Getting Megatron tokenizer for pretrained model name: megatron-gpt-345m, custom vocab file: /tmp/tmpg7tefhb2/bfcdca5e44814366bdb5dcd651325152_gpt2-vocab.json, and merges file: /tmp/tmpg7tefhb2/315a11fd68be49d6abdb34363e8c4997_gpt2-merge.txt
[NeMo I 2023-05-27 09:46:39 tokenizer_utils:130] Getting HuggingFace AutoTokenizer with pretrained_model_name: gpt2, vocab_file: /tmp/tmpg7tefhb2/bfcdca5e44814366bdb5dcd651325152_gpt2-vocab.json, merges_files: /tmp/tmpg7tefhb2/315a11fd68be49d6abdb34363e8c4997_gpt2-merge.txt, special_tokens_dict: {}, and use_fast: False


Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using pad_token, but it is not set yet.
Using mask_token, but it is not set yet.


[NeMo I 2023-05-27 09:46:41 megatron_base_model:238] Padded vocab_size: 50304, original vocab_size: 50257, dummy tokens: 47.
[NeMo I 2023-05-27 09:46:41 megatron_gpt_peft_models:56] Before adding PEFT params:
      | Name  | Type     | Params
    -----------------------------------
    0 | model | GPTModel | 354 M 
    -----------------------------------
    354 M     Trainable params
    0         Non-trainable params
    354 M     Total params
    1,419.485 Total estimated model params size (MB)
[NeMo I 2023-05-27 09:46:41 megatron_gpt_peft_models:65] After adding PEFT params:
      | Name  | Type     | Params
    -----------------------------------
    0 | model | GPTModel | 358 M 
    -----------------------------------
    358 M     Trainable params
    0         Non-trainable params
    358 M     Total params
    1,432.068 Total estimated model params size (MB)
[NeMo I 2023-05-27 09:46:42 nlp_overrides:491] Model MegatronGPTLoRAModel was successfully restored from /home/adithyare

In [21]:
# Training set to 2 epochs by default in a cell above
trainer.fit(model)

[NeMo W 2023-05-27 09:46:51 nemo_logging:349] /home/adithyare/miniconda3/envs/n22/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:175: UserWarning: The `batch_idx` argument in `MegatronGPTLoRAModel.on_train_batch_start` hook may not match with the actual batch index when using a `dataloader_iter` argument in your `training_step`.
      rank_zero_warn(
    
[NeMo W 2023-05-27 09:46:51 nemo_logging:349] /home/adithyare/miniconda3/envs/n22/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:175: UserWarning: The `batch_idx` argument in `MegatronGPTLoRAModel.on_train_batch_end` hook may not match with the actual batch index when using a `dataloader_iter` argument in your `training_step`.
      rank_zero_warn(
    
[NeMo W 2023-05-27 09:46:51 nemo_logging:349] /home/adithyare/miniconda3/envs/n22/lib/python3.8/site-packages/lightning_fabric/plugins/environments/torchelastic.py:36: UserWarning: MASTER_ADDR environment variable is n

[NeMo I 2023-05-27 09:46:51 megatron_gpt_sft_model:634] Building GPT SFT validation datasets.
[NeMo I 2023-05-27 09:46:51 text_memmap_dataset:104] Building data files
[NeMo I 2023-05-27 09:46:51 text_memmap_dataset:343] Processing 1 data files using 12 workers
[NeMo I 2023-05-27 09:46:51 text_memmap_dataset:349] Time building 0 / 1 mem-mapped files: 0:00:00.312421
[NeMo I 2023-05-27 09:46:51 text_memmap_dataset:114] Loading data files
[NeMo I 2023-05-27 09:46:51 text_memmap_dataset:205] Loading data/SQuAD/squad_short_val.jsonl
[NeMo I 2023-05-27 09:46:51 text_memmap_dataset:117] Time loading 1 mem-mapped files: 0:00:00.001111
[NeMo I 2023-05-27 09:46:51 text_memmap_dataset:121] Computing global indices
[NeMo I 2023-05-27 09:46:51 megatron_gpt_sft_model:637] Length of val dataset: 20
[NeMo I 2023-05-27 09:46:51 megatron_gpt_sft_model:648] Building GPT SFT traing datasets.
[NeMo I 2023-05-27 09:46:51 text_memmap_dataset:104] Building data files
[NeMo I 2023-05-27 09:46:51 text_memmap_dat

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


[NeMo I 2023-05-27 09:46:52 nlp_overrides:124] Configuring DDP for model parallelism.
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kqv_adapter
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kqv_adapter
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kqv_adapter
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kqv_adapter
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kqv_adapter
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kqv_adapter
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kqv_adapter
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kqv_adapter
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kqv_adapter
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kqv_adapter
[NeMo I 2023-05-27 09:46:52 adapter_mixins:430] Unfrozen adapter : lora_kq


  | Name  | Type     | Params
-----------------------------------
0 | model | GPTModel | 358 M 
-----------------------------------
3.1 M     Trainable params
354 M     Non-trainable params
358 M     Total params
716.034   Total estimated model params size (MB)


...

[NeMo W 2023-05-27 09:46:52 nemo_logging:349] /home/adithyare/miniconda3/envs/n22/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
      rank_zero_warn(
    
[NeMo W 2023-05-27 09:46:52 nemo_logging:349] /home/adithyare/miniconda3/envs/n22/lib/python3.8/site-packages/pytorch_lightning/loops/dataloader/evaluation_loop.py:401: UserWarning: Found `dataloader_iter` argument in the `validation_step`. Note that the support for this signature is experimental and the behavior is subject to change.
      rank_zero_warn(
    
[NeMo W 2023-05-27 09:46:52 nemo_logging:349] /home/adithyare/miniconda3/envs/n22/lib/python3.8/site-packages/apex/transformer/pipeline_parallel/utils.py:81: UserWarni

Training: 0it [00:00, ?it/s]

[NeMo W 2023-05-27 09:46:55 nemo_logging:349] /home/adithyare/miniconda3/envs/n22/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:232: UserWarning: You called `self.log('global_step', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
      warning_cache.warn(
    
[NeMo W 2023-05-27 09:46:55 nemo_logging:349] /home/adithyare/miniconda3/envs/n22/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:232: UserWarning: You called `self.log('consumed_samples', ...)` in your `training_step` but the value needs to be floating point. Converting it to torch.float32.
      warning_cache.warn(
    
[NeMo W 2023-05-27 09:46:56 nemo_logging:349] /home/adithyare/miniconda3/envs/n22/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:139: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call the

Validation: 0it [00:00, ?it/s]

Epoch 0, global step 100: 'validation_loss' reached 0.30823 (best 0.30823), saving model to '/home/adithyare/NeMo/tutorials/nlp/training_info/checkpoints/lora_example_tuning--validation_loss=0.308-step=100-consumed_samples=396.0.ckpt' as top 1
Metric val_loss improved. New best score: 0.308
`Trainer.fit` stopped: `max_steps=100` reached.
Restoring states from the checkpoint path at /home/adithyare/NeMo/tutorials/nlp/training_info/checkpoints/lora_example_tuning--validation_loss=0.308-step=100-consumed_samples=396.0.ckpt
Restored all states from the checkpoint file at /home/adithyare/NeMo/tutorials/nlp/training_info/checkpoints/lora_example_tuning--validation_loss=0.308-step=100-consumed_samples=396.0.ckpt


In [25]:
# The trained '.nemo' model is saved in the location below:
! ls {config.exp_manager.explicit_log_dir}/checkpoints

 lora_example_tuning.nemo
'lora_example_tuning--validation_loss=0.308-step=100-consumed_samples=396.0.ckpt'
'lora_example_tuning--validation_loss=0.308-step=100-consumed_samples=396.0-last.ckpt'


### Inference
One way to run inference after p-tuning or prompt-tuning your model is to call `model.generate()`. `model.generate()` takes in 

- `inputs` which can be either a list of dictionary objects or `.jsonl` files containing dictionary objects, 
- `length_params`
- `sampling_params`

as arguments. More information about the [text generation API can be found here](https://github.com/NVIDIA/NeMo/blob/main/nemo/collections/nlp/modules/common/transformer/text_generation.py).

If `length_params` and `sampling_params` are set to `None`, the model generates output with a greedy decoding strategy and generates up to `30` new tokens. Most predictive downstream tasks (not text generation tasks), use greedy sampling. To see other ways to run inference with your prompt learning model and more details on how to define various inference parameters, visit `examples/nlp/language_modeling/megatron_gpt_eval.py`.

Below are some randomly selected test examples from the sentiment classification and intent and slot classification test files. Notice that the `label` field is dropped from all test examples. The `MegatronPromptLearningDataset` called within `.generate()` automatically leaves fields in the prompt template empty when they are not provided in the data json. 

In [ ]:
test_examples = [
    {"taskname": "squad", "context": "The build was released for download later in the day in standard 32-bit and 64-bit versions, plus a special 64-bit version which included SDKs and developer tools (Visual Studio Express and Expression Blend) for developing Metro-style apps. The Windows Store was announced during the presentation, but was not available in this build. According to Microsoft, there were about 535,000 downloads of the developer preview within the first 12 hours of its release. Originally set to expire on March 11, 2012, in February 2012 the Developer Preview's expiry date was changed to January 15, 2013.", "question": "When was the Developer preview initially intended to expire?"},
    {"taskname": "squad", "context": "The structures of most federal governments incorporate mechanisms to protect the rights of component states. One method, known as 'intrastate federalism', is to directly represent the governments of component states in federal political institutions. Where a federation has a bicameral legislature the upper house is often used to represent the component states while the lower house represents the people of the nation as a whole. A federal upper house may be based on a special scheme of apportionment, as is the case in the senates of the United States and Australia, where each state is represented by an equal number of senators irrespective of the size of its population.", "question": "What is a bicameral legislature?"},
    {"taskname": "squad", "context": "Imported mystery religions, which offered initiates salvation in the afterlife, were a matter of personal choice for an individual, practiced in addition to carrying on one's family rites and participating in public religion. The mysteries, however, involved exclusive oaths and secrecy, conditions that conservative Romans viewed with suspicion as characteristic of \"magic\", conspiratorial (coniuratio), or subversive activity. Sporadic and sometimes brutal attempts were made to suppress religionists who seemed to threaten traditional morality and unity, as with the senate's efforts to restrict the Bacchanals in 186 BC.", "question": "What was the practice of religion to the Romans?"}
]

In [ ]:
response = model.generate(inputs=test_examples, length_params=None)

print('The prediction results of some sample queries with the trained model:')
for result in response['sentences']:
    print(result)
    print("-" * 30)